In [6]:
using OpenCL

In [7]:
using PyPlot

## OpenCL GPU Mandelbrot Fractal Implementation

In [8]:
mandel_source = "
#pragma OPENCL EXTENSION cl_khr_byte_addressable_store : enable
__kernel void mandelbrot(__global float2 *q,
                         __global ushort *output, 
                         ushort const maxiter)
{
 int gid = get_global_id(0);
 float nreal, real = 0;
 float imag = 0;
 output[gid] = 0;
 for(int curiter = 0; curiter < maxiter; curiter++) {
     nreal = real*real - imag*imag + q[gid].x;
     imag = 2* real*imag + q[gid].y;
     real = nreal;

     if (real*real + imag*imag > 4.0f)
         output[gid] = curiter;
  }
}";

In [31]:
function mandel_opencl(q::Array{Complex64}, maxiter::Int64, device)
    ctx   = cl.Context(device)
    queue = cl.CmdQueue(ctx)

    out = Array{UInt16}(size(q))

    q_buff = cl.Buffer(Complex64, ctx, (:r, :copy), hostbuf=q)
    o_buff = cl.Buffer(UInt16, ctx, :w, length(out))

    prg = cl.Program(ctx, source=mandel_source) |> cl.build!
    
    k = cl.Kernel(prg, "mandelbrot")
    cl.call(queue, k, length(out), nothing, q_buff, o_buff, uint16(maxiter))
    cl.copy!(queue, out, o_buff)
    
    return out
end

mandel_opencl (generic function with 2 methods)

In [32]:
w = 2048 * 2;
h = 2048 * 2;
@printf("Size %i MB\n", sizeof(Complex64) * w * h / 1024 / 1024)

q = [Complex64(r,i) for i=1:-(2.0/w):-1, r=-1.5:(3.0/h):1.5];

Size 128 MB


In [33]:
y1 = -1.0
y2 = 1.0
x1 = -1.5
x2 = 0.5

q = Array{Complex64}(h, w)
for x in 1:w
    for y in 1:h
        xx = x1 + x * ((x2 - x1) / w)
        yy = y1 + y * ((y2 - y1) / h)
        @inbounds q[y, x] = Complex64(xx, yy)
    end
end

In [34]:
device = cl.devices()[3]

OpenCL.Device(GeForce GT 650M on Apple @0x0000000001022700)

In [35]:
@time m = mandel_opencl(q, 30, device);

LoadError: [91mUndefVarError: call not defined[39m

In [36]:
imshow(m, cmap="RdGy")

LoadError: [91mUndefVarError: m not defined[39m